# Game0
Bare bones game.  
* variable board dimensions (w,h)
* variable winning connection length (n)

In [87]:
import itertools
import numpy as np
from IPython.display import clear_output
from sys import exit

## Utility Functions


In [92]:
# count connections to the left and right of pos
def cnct_count(vec, pos):
    count, player = 1, vec[pos]
    for v in vec[:pos][::-1]:
        if v==player: count += 1
        else: break
    for v in vec[pos+1:]:
        if v==player: count += 1
        else: break
    return count

# extract backward diagonal from a nested list
def fdiag(board, w, h, shift=0):
    if shift==0: return [board[k,k] for k in range(min(h,w))]
    elif shift>0: return [board[k,k+shift] for k in range(min(h,w-shift))]
    return [board[k-shift,k] for k in range(min(h+shift,w))]       
    
# extract backward diagonal from a nested list
def bdiag(board, w, h, shift=0):
    if shift==0: return [board[k,w-k-1] for k in range(min(h,w))]
    if shift>0: return [board[k,w-(k+shift)-1] for k in range(min(h,w-shift))]
    return [board[k-shift,w-k-1] for k in range(min(h+shift,w))]   

# check if a value is an integer
def is_int(value):
    try: int(value);   return True
    except: return False

In [136]:
class game0(object):
    def __init__(self, w, h, n, init_board=None):       
        self.w, self.h, self.n = w, h, n
        self.board = np.array([[-1]*w]*h)
        self.player = 0
        self.feasible_moves = list(range(w))
        self.col_count = [0]*w
        self.status = 'active'
               
    # display the status of the board
    def print_board(self):
        clear_output() # comment out for debugging
        for k in range(self.h):
            row = ' '.join(str(r) for r in self.board[k,:])
            row = row.replace('-1','.').replace('1','x')
            print('| ' + row + ' |')
        print('-'*(3+2*self.w))
        print('  ' + ' '.join(str(col) for col in range(self.w)))        

    # get user's input
    def col_input(self):
            
        # ask for and validate input
        while True:
            col = input('Player ' + str(self.player) + ': Enter the column to place your piece ' + 
                        '(' + str(self.player).replace('1','+') + ') ' )
            if col.lower()=='q': print('Thanks for playing!');   exit(0)
            if col=='`': col = 0 # define an alias for 0

            # check for valid input
            if is_int(col): 
                col = int(col)
                if col>=0 and col<self.w and col in self.feasible_moves: break
        return col
        
    # check intersecting row, col, diagonals for connect-n
    def check_col(self, row, col):
        return cnct_count(self.board[:, col], row) >= self.n
    
    def check_row(self, row, col):
        return cnct_count(self.board[row, :], col) >= self.n
    
    def check_fdiag(self, row, col):
        forward_diag = fdiag(self.board, self.w, self.h, col-row)
        return cnct_count(forward_diag, min(row, col)) >= self.n
        
    def check_bdiag(self, row, col):
        col = self.w-1-col
        backward_diag = bdiag(self.board, self.w, self.h, col-row)
        return cnct_count(backward_diag, min(row, col)) >= self.n
    
    # update game        
    def update_game(self, col):
        
        # update col_count, board, player and feasible moves
        self.col_count[col] += 1
        row = self.h-self.col_count[col]
        self.board[row, col] = self.player       
        self.player = 1*(self.player==0)
        if self.col_count[col]==self.n: self.feasible_moves.remove(col)
            
        # check if game is won
        if (self.check_col(row, col) or self.check_row(row, col) or
            self.check_fdiag(row, col) or self.check_bdiag(row, col)):
            self.status = 'won';   return
        
        # check for draws
        if not self.feasible_moves: self.status = 'draw'

    # two-player game
    def play_game(self):
        while True:
            self.print_board()
            col = self.col_input()
            self.update_game(col)
            if self.status!='active':
                self.print_board()
                if self.status=='won': print('Player ' + str(1-self.player) + ' won!')
                else: print('Game is a draw.')
                return            

In [137]:
g = game0(w=3,h=3,n=3)
g.play_game()

| 0 x . |
| x 0 . |
| 0 x 0 |
---------
  0 1 2
Player 0 won!


# game1
* add special characters
* takes intital board configuration as input
* update print function for special characters

In [133]:
class game1(game0):
    def __init__(self, w, h, n, init_board=None):
        # set up initial board
        if init_board is not None: 
            self.board = init_board
            self.w, self.h = init_board.shape
        else: self.board = np.zeros((self.h, self.w), dtype=np.int)-1
        self.col_capacity = np.sum(self.board>-2, axis=0)        
        
        # add game0 attributes
        game0.__init__(self, self.w, self.h, n)
               
    # display the status of the board
    def print_board(self):
        clear_output() # comment out for debugging
        for k in range(self.h):
            row = ' '.join(str(r) for r in self.board[k,:])
            row = row.replace('-3', '?').replace('-2', '#').replace('-1','.').replace('1','x')
            print('| ' + row + ' |')
        print('-'*(3+2*self.w))
        print('  ' + ' '.join(str(col) for col in range(self.w)))        
        

    # update game        
    def update_game(self, col):
        
        # update col_count and row
        self.col_count[col] += 1
        row = self.h-self.col_count[col]
        while self.board[row, col]<-1: row -= 1 # skip special pieces

        #update board, player and feasible moves
        self.board[row, col] = self.player       
        self.player = 1*(self.player==0)
        if self.col_count[col] == self.col_capacity[col]: self.feasible_moves.remove(col)
            
        # check if game is over
        if (game0.check_col(self, row, col) or game0.check_row(self, row, col) or
            game0.check_fdiag(self, row, col) or game0.check_bdiag(self, row, col)):
            self.status = 'won';   return 
            
        # check for draws
        if not self.feasible_moves: self.status = 'draw'


In [132]:
init_board = np.zeros((4, 4), dtype=np.int)-1
init_board[1,0] = -2

g = game1(w=3,h=3,n=3, init_board=init_board)
#g.play_game()

[[-1 -1 -1 -1]
 [-2 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]]
(4, 4)


# Extra Stuff

In [ ]:
# check if a value is being used as a special piece
special_pieces = ['@','.', '#', "?", '|']
def is_special(v): return v in special_pieces


        """
        # ask for player's info
        self.name = input('Please enter your name: ')
        self.pc = input('Please enter the game piece that you would like to use: ')
        while is_special(self.pc):
            clear_output()
            print('Sorry. That piece is already in use.')
            self.pc = input('Please enter any piece other than: ' + str(special_pieces).strip("[]"))        
        """